In [ ]:
from elemeno_ai_sdk.ml.features.feature_store import FeatureStore
import pandas as pd
import os

In [ ]:
os.environ["MLHUB_API_KEY"] = "ADD YOUR API KEY HERE"

In [ ]:
raw_data = pd.read_csv("./example_data/sample.csv")
raw_data.head()

In [ ]:
raw_data.shape

In [ ]:
fs = FeatureStore()

In [ ]:
await fs.ingest(feature_table_name="creditcard_demo_v5", to_ingest=raw_data)

In [ ]:
data = await fs.get_training_features(
    feature_table_name="creditcard_demo_v5",
    date_from="2022-01-01", 
    date_to="2023-10-04"
)

In [ ]:
data